In [2]:
pip install scikit-learn pandas numpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [ ]:
# Loading the dataset (products.csv)
df = pd.read_csv('C:\\Users\\acer\\Desktop\\Infinity AI Work\\InfinityStyleVerse\\data\\raw\\products.csv')

In [15]:
df.head()

,title,category,color,material,rating,description
0,Slim Fit Blazer,Formal,Navy,Cotton,4.6,Stylish modern blazer for professionals.
1,Floral Midi Dress,Dress,Red,Polyester,4.3,Elegant floral dress perfect for summer outings.
2,Denim Jacket,Casual,Blue,Denim,4.7,Classic denim jacket with a relaxed fit.
3,Silk Scarf,Accessories,Green,Silk,4.2,Luxurious silk scarf to elevate any outfit.
4,Leather Ankle Boots,Footwear,Black,Leather,4.8,Sleek leather boots for a chic look.
